<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/KD_DR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install cloud-tpu-client==0.10 torch==1.8.1 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8.1-cp37-cp37m-linux_x86_64.whl tensorboard-plugin-profile

In [2]:
!pip uninstall torch -y
!pip install torch==1.8.1

Found existing installation: torch 1.9.0+cu102
Uninstalling torch-1.9.0+cu102:
  Successfully uninstalled torch-1.9.0+cu102
     |████████████████████████████████| 804.1 MB 2.0 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.


In [1]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import os
import sys
import time
import logging
import argparse
import numpy as np
from itertools import chain

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
import torchvision.datasets as dst
import torchvision

from __future__ import absolute_import
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

from google.colab import auth, drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [58]:
def accuracy(output, target, topk=(1,)):
	"""Computes the precision@k for the specified values of k"""
	maxk = max(topk)
	batch_size = target.size(0)

	_, pred = output.topk(maxk, 1, True, True)
	pred    = pred.t()
	correct = pred.eq(target.view(1, -1).expand_as(pred))

	res = []
	for k in topk:
		correct_k = correct[:k].view(-1).float().sum(0)
		res.append(correct_k.mul_(100.0 / batch_size))
	return res

In [3]:
# Mount Drive
drive.mount('/content/drive')
auth.authenticate_user()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import gdown

def download(data, url):
    # Download dataset
    import zipfile
    url = url
    output = "{}.zip".format(data)
    gdown.download(url, output, quiet=False)

    # Uncompress dataset
    local_zip = '{}.zip'.format(data)
    zip_ref = zipfile.ZipFile(local_zip, "r")
    zip_ref.extractall()
    zip_ref.close()

In [5]:
def get_model_property(key):
  default_model_property = {
      'img_dim': 224,
      'model_fn': getattr(torchvision.models, FLAGS.model)
  }
  model_properties = {
      'inception_v3': {
          'img_dim': 299,
          'model_fn': lambda: torchvision.models.inception_v3(aux_logits=False)
      },
  }
  model_fn = model_properties.get(FLAGS.model, default_model_property)[key]
  return model_fn

In [6]:
data_samples = {
    "sample@200": "https://drive.google.com/uc?id=1FfV7YyDJvNUCDP5r3-8iQfZ2-xJp_pgb",
    "sample@500": "https://drive.google.com/uc?id=1dHwUqpmSogEdjAB9rwDUL-OKFRUcVXte",
    "sample@1000": "https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2",
    "sample@2000": "https://drive.google.com/uc?id=1PB7uGd-dUnZKnKZpZl-HvE1DVcWgX50F",
    "sample@3000": "https://drive.google.com/uc?id=1_yre5K9YYvJgSrT4xvrI8eD_htucIywA",
    "sample@4000_images": "https://drive.google.com/uc?id=1dqVB8EozEpwWzyuU80AauoQmsiw3Gtm2",
    "sample@20000": "https://drive.google.com/uc?id=1MTDpLzpmhSiZq2jSdmHx2UDPn9FC8gzO",
    "val-voets-tf": "https://drive.google.com/uc?id=1VzVgMGTkBBPG2qbzLunD9HvLzH6tcyrv",
    "train_voets": "https://drive.google.com/uc?id=1AmcFh1MOOZ6aqKm2eO7XEdgmIEqHKTZ5",
    "voets_test_images": "https://drive.google.com/uc?id=15S_V3B_Z3BOjCT3AbO2c887FyS5B0Lyd"
}

In [7]:
LABELED = 'sample@1000'

In [8]:
URL_LABELED = data_samples[LABELED]
download(LABELED, URL_LABELED)

Downloading...
From: https://drive.google.com/uc?id=1DPZrHrj3Bdte5Dc6NCZ33CAqMG-Oipa2
To: /content/sample@1000.zip
108MB [00:00, 228MB/s] 


In [9]:
!wget "https://raw.githubusercontent.com/pytorch/xla/master/test/args_parse.py"

--2021-08-22 17:00:16--  https://raw.githubusercontent.com/pytorch/xla/master/test/args_parse.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2036 (2.0K) [text/plain]
Saving to: ‘args_parse.py.1’

args_parse.py.1     100%[===================>]   1.99K  --.-KB/s    in 0s      

2021-08-22 17:00:17 (34.9 MB/s) - ‘args_parse.py.1’ saved [2036/2036]



In [10]:
import args_parse

SUPPORTED_MODELS = [
    'alexnet', 'densenet121', 'densenet161', 'densenet169', 'densenet201',
    'inception_v3', 'resnet101', 'resnet152', 'resnet18', 'resnet34',
    'resnet50', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13',
    'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn'
]

MODEL_OPTS = {
    '--model': {
        'choices': SUPPORTED_MODELS,
        'default': 'resnet50',
    },
    '--test_set_batch_size': {
        'type': int,
    },
    '--lr_scheduler_type': {
        'type': str,
    },
    '--lr_scheduler_divide_every_n_epochs': {
        'type': int,
    },
    '--lr_scheduler_divisor': {
        'type': int,
    },
    '--test_only_at_end': {
        'action': 'store_true',
    },
}

In [11]:
FLAGS = args_parse.parse_common_options(
    datadir=LABELED,
    batch_size=None,
    num_epochs=None,
    momentum=None,
    lr=None,
    target_accuracy=None,
    profiler_port=9012,
    opts=MODEL_OPTS.items(),
)

In [12]:
EPOCHS = 200

In [13]:
FLAGS.fake_data = False
FLAGS.num_epochs = EPOCHS
FLAGS.batch_size = 64
FLAGS.log_steps = 100
FLAGS.num_cores = 8

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
model1 = get_model_property('model_fn')().to(device)

model1.fc = nn.Sequential(
      nn.Linear(2048, 512),
      nn.Linear(512, 1),
      nn.Sigmoid()
  ).to(device)

In [16]:
state_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/SimCLR/models/SimCLR-1-DR-pytorch/net-DR-SimCLR-Finetuned-Test-{}.pt'.format(LABELED))

In [17]:
state_dict.keys()

dict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.wei

In [18]:
log = model1.load_state_dict(state_dict, strict=False)
print(log)

<All keys matched successfully>


In [19]:
model2 = get_model_property('model_fn')().to(device)

model2.fc = nn.Sequential(
      nn.Linear(2048, 512),
      nn.Linear(512, 1),
      nn.Sigmoid()
  ).to(device)

In [20]:
log = model2.load_state_dict(state_dict, strict=False)
print(log)

<All keys matched successfully>


In [21]:
"""
TO DO:
------

  Check if las terminaciones del model1 y model2 estan OK, es decir si tienen sentido,
  con lo del error de mas abajo. 
"""

'\nTO DO:\n------\n\n  Check if las terminaciones del model1 y model2 estan OK, es decir si tienen sentido,\n  con lo del error de mas abajo. \n'

In [22]:
parser = argparse.ArgumentParser(description='train kd')

# various path
parser.add_argument('--save_root', type=str, default='./results', help='models and logs are saved here')
parser.add_argument('--img_root', type=str, default='./datasets', help='path name of image dataset')
parser.add_argument('--s_init', type=str, required=True, help='initial parameters of student model')
parser.add_argument('--t_model', type=str, required=True, help='path name of teacher model')

# training hyper parameters
parser.add_argument('--print_freq', type=int, default=50, help='frequency of showing training results on console')
parser.add_argument('--epochs', type=int, default=200, help='number of total epochs to run')
parser.add_argument('--batch_size', type=int, default=128, help='The size of batch')
parser.add_argument('--lr', type=float, default=0.1, help='initial learning rate')
parser.add_argument('--momentum', type=float, default=0.9, help='momentum')
parser.add_argument('--weight_decay', type=float, default=1e-4, help='weight decay')
parser.add_argument('--num_class', type=int, default=10, help='number of classes')
parser.add_argument('--cuda', type=int, default=1)

# others
parser.add_argument('--seed', type=int, default=2, help='random seed')
parser.add_argument('--note', type=str, default='try', help='note for this run')

# net and dataset choosen
parser.add_argument('--data_name', type=str, required=True, help='name of dataset') # cifar10/cifar100
parser.add_argument('--t_name', type=str, required=True, help='name of teacher')    # resnet20/resnet110
parser.add_argument('--s_name', type=str, required=True, help='name of student')    # resnet20/resnet110

# hyperparameter
parser.add_argument('--kd_mode', type=str, required=True, help='mode of kd, which can be:'
															   'logits/st/at/fitnet/nst/pkt/fsp/rkd/ab/'
															   'sp/sobolev/cc/lwm/irg/vid/ofd/afd')
parser.add_argument('--lambda_kd', type=float, default=1.0, help='trade-off parameter for kd loss')
parser.add_argument('--T', type=float, default=4.0, help='temperature for ST')
parser.add_argument('--p', type=float, default=2.0, help='power for AT')
parser.add_argument('--w_dist', type=float, default=25.0, help='weight for RKD distance')
parser.add_argument('--w_angle', type=float, default=50.0, help='weight for RKD angle')
parser.add_argument('--m', type=float, default=2.0, help='margin for AB')
parser.add_argument('--gamma', type=float, default=0.4, help='gamma in Gaussian RBF for CC')
parser.add_argument('--P_order', type=int, default=2, help='P-order Taylor series of Gaussian RBF for CC')
parser.add_argument('--w_irg_vert', type=float, default=0.1, help='weight for IRG vertex')
parser.add_argument('--w_irg_edge', type=float, default=5.0, help='weight for IRG edge')
parser.add_argument('--w_irg_tran', type=float, default=5.0, help='weight for IRG transformation')
parser.add_argument('--sf', type=float, default=1.0, help='scale factor for VID, i.e. mid_channels = sf * out_channels')
parser.add_argument('--init_var', type=float, default=5.0, help='initial variance for VID')
parser.add_argument('--att_f', type=float, default=1.0, help='attention factor of mid_channels for AFD')


_StoreAction(option_strings=['--att_f'], dest='att_f', nargs=None, const=None, default=1.0, type=<class 'float'>, choices=None, help='attention factor of mid_channels for AFD', metavar=None)

In [23]:
args, unparsed = parser.parse_known_args(['--save_root', './results/logits/',
                                          '--t_model', './results/base/base-c10-r110/model_best.pth.tar',
                                          '--s_init', './results/base/base-c10-r20/initial_r20.pth.tar',
                                          '--data_name', 'DR',
                                          '--batch_size', '32',
                                          '--t_name', 'resnet110', 
                                          '--s_name', 'resnet20',
                                          '--num_class', '2',
                                          '--kd_mode', 'logits', 
                                          '--lambda_kd', '0.1',
                                          '--note', 'logits-c10-r110-r20'
                                          ])

In [24]:
class Logits(nn.Module):
	'''
	Do Deep Nets Really Need to be Deep?
	http://papers.nips.cc/paper/5484-do-deep-nets-really-need-to-be-deep.pdf
	'''
	def __init__(self):
		super(Logits, self).__init__()

	def forward(self, out_s, out_t):
		loss = F.mse_loss(out_s, out_t)

		return loss


In [25]:
def adjust_lr(optimizer, epoch):
	scale   = 0.1
	lr_list =  [args.lr] * 100
	lr_list += [args.lr*scale] * 50
	lr_list += [args.lr*scale*scale] * 50

	lr = lr_list[epoch-1]
	logging.info('Epoch: {}  lr: {:.3f}'.format(epoch, lr))
	for param_group in optimizer.param_groups:
		param_group['lr'] = lr

In [26]:
snet = model1

In [27]:
tnet = model2

In [28]:
for param in tnet.parameters():
  param.requires_grad = False

In [29]:
logging.info('Teacher: %s', tnet)
logging.info('-----------------------------------------------')

In [30]:
criterionKD = Logits()

In [31]:
criterionCls = torch.nn.CrossEntropyLoss().cuda()

In [32]:
optimizer = torch.optim.SGD(snet.parameters(),
              lr = args.lr, 
              momentum = args.momentum, 
              weight_decay = args.weight_decay,
              nesterov = True)

In [33]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0001
)

In [34]:
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2470, 0.2435, 0.2616)

In [35]:
train_transform = transforms.Compose([
    transforms.Pad(4, padding_mode='reflect'),
    transforms.RandomCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,std=std)
  ])

In [36]:
img_dim = get_model_property('img_dim')

In [37]:
normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [38]:
train_dataset = torchvision.datasets.ImageFolder(
    os.path.join(FLAGS.datadir, 'train'),
    transforms.Compose([
        transforms.RandomResizedCrop(img_dim),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

In [39]:
# define data loader
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size, 
    shuffle=True, 
    num_workers=2, 
    pin_memory=True)

In [40]:
# warp nets and criterions for train and test
nets = {'snet':snet, 'tnet':tnet}
criterions = {'criterionCls':criterionCls, 'criterionKD':criterionKD}

In [41]:
############## DEBUG ##############

In [68]:
batch_time = AverageMeter()
data_time  = AverageMeter()
cls_losses = AverageMeter()
kd_losses  = AverageMeter()
top1       = AverageMeter()
top5       = AverageMeter()

NameError: ignored

In [42]:
epoch = 0 

In [43]:
adjust_lr(optimizer, epoch)

In [44]:
epoch_start_time = time.time()

In [45]:
snet = nets['snet']
tnet = nets['tnet']

In [46]:
criterionCls = criterions['criterionCls']
criterionKD  = criterions['criterionKD']

In [47]:
snet.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [65]:
for i, (img, target) in enumerate(train_loader, start=1):
  break

RuntimeError: ignored

In [61]:
if args.cuda:
  img = img.cuda(non_blocking=True)
  target = target.cuda(non_blocking=True)

In [62]:
args.cuda

1

In [63]:
out_s = snet(img)
out_t = tnet(img)

RuntimeError: ignored

In [ ]:
out_s[0]

In [64]:
out_t[0]

RuntimeError: ignored

In [60]:
cls_loss = criterionCls(out_s, target)

RuntimeError: ignored

In [54]:
if args.kd_mode in ['logits', 'st']:
  kd_loss = criterionKD(out_s, out_t.detach()) * args.lambda_kd

In [55]:
kd_loss

tensor(0., device='cuda:0', grad_fn=<MulBackward0>)

In [59]:
loss = cls_loss + kd_loss

RuntimeError: ignored

In [66]:
loss

NameError: ignored

In [ ]:
prec1, prec5 = accuracy(out_s, target, topk=(1,5))

In [67]:
cls_losses.update(cls_loss.item(), img.size(0))
kd_losses.update(kd_loss.item(), img.size(0))

NameError: ignored

In [69]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

NameError: ignored

In [ ]:
f_l = [cls_losses.avg, kd_losses.avg, top1.avg, top5.avg]
logging.info('Cls: {:.4f}, KD: {:.4f}, Prec@1: {:.2f}, Prec@5: {:.2f}'.format(*f_l))


In [ ]:
"""
Idea: 
  - Comparar realizando Knowledge distillation solo de LABELED 1000 
  VS Knowledge distilattion de la UNLABELED 5000 (Solo el loss de consistency) + 
  despues un Finetuning de la Unlabeled
"""

In [ ]:
####################################

In [ ]:
###################################

In [ ]:
best_top1 = 0
best_top5 = 0
for epoch in range(1, args.epochs+1):
  adjust_lr(optimizer, epoch)

  # train one epoch
  epoch_start_time = time.time()
  train(train_loader, nets, optimizer, criterions, epoch)

  # evaluate on testing set
  logging.info('Testing the models......')
  test_top1, test_top5 = test(test_loader, nets, criterions, epoch)

  epoch_duration = time.time() - epoch_start_time
  logging.info('Epoch time: {}s'.format(int(epoch_duration)))

  # save model
  is_best = False
  if test_top1 > best_top1:
    best_top1 = test_top1
    best_top5 = test_top5
    is_best = True
  logging.info('Saving models......')
  save_checkpoint({
    'epoch': epoch,
    'snet': snet.state_dict(),
    'tnet': tnet.state_dict(),
    'prec@1': test_top1,
    'prec@5': test_top5,
  }, is_best, args.save_root)